In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader


In [2]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
data = torch.tensor(iris[iris.columns[0:4]].values).float()

labels = torch.zeros(len(data), dtype=torch.long)


In [4]:
labels[iris.species == 'versicolor'] = 1
labels[iris.species == 'virginica'] = 2

In [5]:
labels.unique()

tensor([0, 1, 2])

In [6]:
train_data,test_data,train_labels,test_labels = train_test_split(data,labels,test_size=.2)

train_dataset = TensorDataset(train_data,train_labels)
test_dataset = TensorDataset(test_data,test_labels)

train_loader = DataLoader(train_dataset,batch_size=64,shuffle = True,drop_last = True)
test_loader = DataLoader(test_dataset,batch_size=test_dataset.tensors[0].shape[0])

In [7]:

def createANewModel():
    ANNiris = nn.Sequential(nn.Linear(4,64),
                           nn.ReLU(),
                           nn.Linear(64,64),
                           nn.ReLU(),
                           nn.Linear(64,3))
    
    lossfun = nn.CrossEntropyLoss()
    
    optimizer = torch.optim.SGD(ANNiris.parameters(),lr=.005)
    
    return ANNiris,lossfun,optimizer

In [9]:
tmpmodel = createANewModel()[0]

print(tmpmodel)

for i in tmpmodel.named_parameters():
    print(i[0],i[1].shape,i[1].numel())

Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=3, bias=True)
)
0.weight torch.Size([64, 4]) 256
0.bias torch.Size([64]) 64
2.weight torch.Size([64, 64]) 4096
2.bias torch.Size([64]) 64
4.weight torch.Size([3, 64]) 192
4.bias torch.Size([3]) 3


In [10]:
numepochs = 1000

def trainTheModel(L1lambda):
    
    trainAcc = []
    testAcc = []
    losses = []
    
    nweight = 0
    
    for pname,weight in ANNiris.named_parameters():
        
        if 'bias' not in pname:
            
            nweights = nweights + weight.numel()
            
    for epochi in range(numepochs):
        
        batchAcc = []
        batchLoss = []
        
        for X,y in train_loader:
            
            yHat = ANNiris(X)
            loss = lossfun(yHat,y)
            
            L1_term = torch.tensor(0.,requires_grad = True)
            
            for pname,weight in ANNiris.names_parameters():
                
                L1_term = L1_term + torch.sum(torch.abs(weight))
                
            loss = loss + L1lambda*L1_term/nweights
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batchAcc.append(100*torch.mean((torch.argmax(yHat,axis = 1)==y).float()).item())
            batchLoss.append(np.mean(batchLoss))
            
            X,y = next(iter(test_loader))
            predlabels = torch.argmax(ANNiris(X),axis = 1 )
            testAcc.append(100*torch.mean((predlabels == y).float()).item())

SyntaxError: invalid syntax (Temp/ipykernel_12968/3456674515.py, line 29)